<a href="https://colab.research.google.com/github/AneVillanueva/03MIAR_Algoritmos-de-optimizacion/blob/main/Algoritmos_Ane_Villanueva_AG3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Actividad Guiada 3

<font color="gray">**Asignatura: Algoritmos de Optimización**

<font color="darkcyan">Alumna: Ane Villanueva Etxebarria

Link repositorio GitHub: https://github.com/AneVillanueva/03MIAR_Algoritmos-de-optimizacion
<br>
<hr>

#Carga de librerias

In [ ]:
!pip install requests    #Hacer llamadas http a paginas de la red
!pip install tsplib95    #Modulo para las instancias del problema del TSP

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 53.8 MB/s eta 0:00:00
  Attempting uninstall: networkx
    Found existing installation: networkx 3.1
    Uninstalling networkx-3.1:
      Successfully uninstalled networkx-3.1


#Carga de los datos del problema

In [ ]:
import urllib.request #Hacer llamadas http a paginas de la red
import tsplib95       #Modulo para las instancias del problema del TSP
import math           #Modulo de funciones matematicas. Se usa para exp
import random         #Para generar valores aleatorios

#Descargamos el fichero de datos (Matriz de distancias)
file = "swiss42.tsp" ;
urllib.request.urlretrieve("http://comopt.ifi.uni-heidelberg.de/software/TSPLIB95/tsp/swiss42.tsp.gz", file + '.gz')
!gzip -d swiss42.tsp.gz     #Descomprimir el fichero de datos



In [ ]:
#Carga de datos y generación de objeto problem
###############################################################################
problem = tsplib95.load(file)

#Nodos
Nodos = list(problem.get_nodes())

#Aristas
Aristas = list(problem.get_edges())



In [ ]:
#Probamos algunas funciones del objeto problem

#Distancia entre nodos
problem.get_weight(0, 1) # Nos dice la distancia entre el nodo 0 y el nodo 1

#Todas las funciones
#Documentación: https://tsplib95.readthedocs.io/en/v0.6.1/modules.html

#dir(problem)

15

#Funcionas basicas


In [ ]:

#Funcionas basicas
###############################################################################

#Se genera una solucion aleatoria con comienzo en en el nodo 0
def crear_solucion(Nodos):
  solucion = [Nodos[0]]
  for n in Nodos[1:]:
    solucion = solucion + [random.choice(list(set(Nodos) - set({Nodos[0]}) - set(solucion)))]
  return solucion

#Devuelve la distancia entre dos nodos
def distancia(a,b, problem):
  return problem.get_weight(a,b)

#Devuelve la distancia total de una trayectoria/solucion
def distancia_total(solucion, problem):
  distancia_total = 0
  for i in range(len(solucion)-1):
    distancia_total += distancia(solucion[i] ,solucion[i+1] ,  problem)
  return distancia_total + distancia(solucion[len(solucion)-1] ,solucion[0], problem)




#BUSQUEDA ALEATORIA

In [ ]:
###############################################################################
# BUSQUEDA ALEATORIA
###############################################################################

def busqueda_aleatoria(problem, N):
  #N es el numero de iteraciones
  Nodos = list(problem.get_nodes())

  mejor_solucion = []
  #mejor_distancia = 10e100                         #Inicializamos con un valor alto
  mejor_distancia = float('inf')                    #Inicializamos con un valor alto

  for i in range(N):                                #Criterio de parada: repetir N veces pero podemos incluir otros
    solucion = crear_solucion(Nodos)                #Genera una solucion aleatoria
    distancia = distancia_total(solucion, problem)  #Calcula el valor objetivo (distancia total)

    if distancia < mejor_distancia:                 #Compara con la mejor obtenida hasta ahora
      mejor_solucion = solucion
      mejor_distancia = distancia


  print("Mejor solución:" , mejor_solucion)
  print("Distancia     :" , mejor_distancia)
  return mejor_solucion


#Busqueda aleatoria con 5000 iteraciones
solucion = busqueda_aleatoria(problem, 10000)

Mejor solución: [0, 11, 25, 21, 24, 9, 30, 22, 1, 41, 20, 38, 40, 23, 39, 8, 34, 6, 15, 16, 36, 31, 29, 3, 14, 35, 17, 37, 4, 28, 18, 10, 5, 19, 7, 27, 33, 32, 2, 26, 13, 12]
Distancia     : 3378


#BUSQUEDA LOCAL

In [ ]:
###############################################################################
# BUSQUEDA LOCAL
###############################################################################
def genera_vecina(solucion):
  #Generador de soluciones vecinas: 2-opt (intercambiar 2 nodos) Si hay N nodos se generan (N-1)x(N-2)/2 soluciones
  #Se puede modificar para aplicar otros generadores distintos que 2-opt
  #print(solucion)
  mejor_solucion = []
  mejor_distancia = 10e100
  for i in range(1,len(solucion)-1):          #Recorremos todos los nodos en bucle doble para evaluar todos los intercambios 2-opt
    for j in range(i+1, len(solucion)):

      #Se genera una nueva solución intercambiando los dos nodos i,j:
      #  (usamos el operador + que para listas en python las concatena) : ej.: [1,2] + [3] = [1,2,3]
      vecina = solucion[:i] + [solucion[j]] + solucion[i+1:j] + [solucion[i]] + solucion[j+1:]

      #Se evalua la nueva solución ...
      distancia_vecina = distancia_total(vecina, problem)

      #... para guardarla si mejora las anteriores
      if distancia_vecina <= mejor_distancia:
        mejor_distancia = distancia_vecina
        mejor_solucion = vecina
  return mejor_solucion


#solucion = [1, 47, 13, 41, 40, 19, 42, 44, 37, 5, 22, 28, 3, 2, 29, 21, 50, 34, 30, 9, 16, 11, 38, 49, 10, 39, 33, 45, 15, 24, 43, 26, 31, 36, 35, 20, 8, 7, 23, 48, 27, 12, 17, 4, 18, 25, 14, 6, 51, 46, 32]
print("Distancia Solucion Incial:" , distancia_total(solucion, problem))


nueva_solucion = genera_vecina(solucion)
print("Distancia Mejor Solucion Local:", distancia_total(nueva_solucion, problem))


Distancia Solucion Incial: 3378
Distancia Mejor Solucion Local: 3089


In [ ]:
#Busqueda Local:
#  - Sobre el operador de vecindad 2-opt(funcion genera_vecina)
#  - Sin criterio de parada, se para cuando no es posible mejorar.
def busqueda_local(problem):
  mejor_solucion = []

  #Generar una solucion inicial de referencia(aleatoria)
  solucion_referencia = crear_solucion(Nodos)
  mejor_distancia = distancia_total(solucion_referencia, problem)
  print('Distancia de la solución inicial: ', mejor_distancia)
  iteracion=0             #Un contador para saber las iteraciones que hacemos
  while(1):
    iteracion +=1         #Incrementamos el contador
    #print('#',iteracion)

    #Obtenemos la mejor vecina ...
    vecina = genera_vecina(solucion_referencia)

    #... y la evaluamos para ver si mejoramos respecto a lo encontrado hasta el momento
    distancia_vecina = distancia_total(vecina, problem)

    #Si no mejoramos hay que terminar. Hemos llegado a un minimo local(según nuestro operador de vecindad 2-opt)
    if distancia_vecina < mejor_distancia:
      #mejor_solucion = copy.deepcopy(vecina)   #Con copia profunda. Las copias en python son por referencia
      mejor_solucion = vecina                   #Guarda la mejor solución encontrada
      mejor_distancia = distancia_vecina

    else:
      print("En la iteracion ", iteracion, ", la mejor solución encontrada es:" , mejor_solucion)
      print("Distancia     :" , mejor_distancia)
      return mejor_solucion

    solucion_referencia = vecina


sol = busqueda_local(problem )

Distancia de la solución inicial:  4760
En la iteracion  33 , la mejor solución encontrada es: [0, 27, 3, 1, 7, 17, 37, 15, 16, 14, 6, 4, 2, 28, 34, 33, 20, 32, 19, 13, 5, 26, 18, 10, 8, 29, 30, 39, 21, 24, 40, 23, 12, 11, 25, 41, 9, 22, 38, 35, 36, 31]
Distancia     : 1789


#BUSQUEDA LOCAL - Propuesta de mejora

> <font color="gray"> Volver a comenzar con otras soluciones iniciales > **búsquedas multi-arranque**

En este caso, en vez de detener el algoritmo cuando la solución no mejora, se ha modificado el código para que trate de buscar mejores soluciones partiendo de una solución inicial diferente.

Para ello, se ha establecido un tiempo máximo. Mientras este tiempo máximo no se supere, la función aplicará el algoritmo de búsqueda local proporcionado por el profesor, partiendo de diferentes soluciones iniciales, devolviendo la solución correspondiente en cada llamada. Finalmente, la mejora aplicada, devolverá la mejor solución de todas las que se hayan calculado.

Para analizar la mejora del algoritmo, se han analizado los resultados obtenidos para diferentes tiempos: se han analizado los resultados obtenidos con 30 segundos, 1 minuto, 2 minutos y 5 minutos.

In [ ]:
def busqueda_local_v1(problem):
  mejor_solucion = []
  solucion_referencia = crear_solucion(Nodos)
  mejor_distancia = distancia_total(solucion_referencia, problem)

  iteracion=0
  while(1):
    iteracion +=1

    vecina = genera_vecina(solucion_referencia)
    distancia_vecina = distancia_total(vecina, problem)

    if distancia_vecina < mejor_distancia:
      mejor_solucion = vecina
      mejor_distancia = distancia_vecina
      solucion_referencia = vecina

    # Si no mejoramos, paramos
    else:
      return mejor_solucion

In [ ]:
import time

def busqueda_local_mejora1(t_max):
  start_time = time.time()
  it = 0
  mejor_distancia = float('inf')
  while  (time.time() - start_time) < t_max:
    it += 1
    # Calculamos soluciones
    sol = busqueda_local_v1(problem)
    dist_sol = distancia_total(sol, problem)
    if dist_sol < mejor_distancia:
      mejor_solucion = sol
      mejor_distancia = dist_sol

  print('Iteraciones realizadas: ', it)
  print('Mejor distancia obtenida: ', mejor_distancia)
  print('Mejor solución: ', mejor_solucion)

In [ ]:
# Solución en 30 segundos:
busqueda_local_mejora1(30)

Iteraciones realizadas:  9
Mejor distancia obtenida:  1582
Mejor solución:  [0, 30, 38, 22, 2, 4, 6, 7, 15, 37, 17, 31, 32, 34, 33, 20, 35, 36, 16, 14, 19, 13, 5, 26, 18, 12, 11, 25, 10, 41, 23, 8, 9, 21, 40, 24, 39, 29, 28, 27, 3, 1]


In [ ]:
# Solución en 1 minuto:
busqueda_local_mejora1(60)

Iteraciones realizadas:  16
Mejor distancia obtenida:  1533
Mejor solución:  [0, 28, 29, 30, 38, 22, 9, 8, 41, 23, 21, 40, 24, 39, 10, 25, 11, 12, 18, 26, 4, 3, 2, 27, 32, 34, 33, 20, 31, 17, 7, 1, 6, 5, 13, 19, 14, 16, 15, 37, 36, 35]


In [ ]:
# Solución en 2 minutos:
busqueda_local_mejora1(120)

Iteraciones realizadas:  36
Mejor distancia obtenida:  1571
Mejor solución:  [0, 27, 2, 3, 4, 1, 7, 17, 31, 36, 35, 32, 30, 10, 25, 11, 12, 18, 13, 19, 14, 16, 15, 37, 20, 33, 34, 38, 22, 39, 24, 40, 21, 9, 23, 41, 8, 29, 28, 26, 5, 6]


In [ ]:
# Solución en 5 minutos:
busqueda_local_mejora1(300)

Iteraciones realizadas:  86
Mejor distancia obtenida:  1465
Mejor solución:  [0, 32, 31, 17, 36, 35, 20, 33, 34, 38, 22, 39, 24, 40, 21, 9, 23, 41, 10, 25, 11, 12, 18, 6, 4, 2, 28, 27, 3, 1, 7, 37, 15, 16, 14, 19, 13, 5, 26, 8, 29, 30]


In [ ]:
# Solución en 10 minutos:
busqueda_local_mejora1(600)

Iteraciones realizadas:  175
Mejor distancia obtenida:  1508
Mejor solución:  [0, 1, 6, 11, 25, 41, 40, 24, 21, 39, 22, 38, 30, 29, 9, 23, 8, 10, 12, 18, 26, 5, 14, 16, 15, 37, 17, 31, 36, 35, 20, 33, 34, 32, 28, 27, 2, 3, 4, 13, 19, 7]
